![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [8]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
import os

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# filename_features = "../Dados/UCI HAR Dataset/features.txt"
# filename_labels = "../Dados/UCI HAR Dataset/activity_labels.txt"

# filename_subtrain = "../Dados/UCI HAR Dataset/train/subject_train.txt"
# filename_xtrain = "../Dados/UCI HAR Dataset/train/X_train.txt"
# filename_ytrain = "../Dados/UCI HAR Dataset/train/y_train.txt"

# filename_subtest = "../Dados/UCI HAR Dataset/test/subject_test.txt"
# ffilename_xtest = "../Dados/UCI HAR Dataset/test/X_test.txt"
# filename_ytest = "../Dados/UCI HAR Dataset/test/y_test.txt"

# features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
# labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

# subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
# X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
# y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

# subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
# X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
# y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

In [36]:
#teste:
def processa_dados_leitura_e_salvamento(caminho_features, caminho_subject, caminho_X, tipo='train'):
   # Carregar features.txt em uma Series
   features = pd.read_csv(caminho_features, delim_whitespace=True, header=None, usecols=[1]).squeeze()
   features.name = 'Var'

   # Carregar subject_train.txt ou subject_test.txt em uma Series
   subject = pd.read_csv(caminho_subject, header=None).squeeze()
   subject.name = 'subject'

   # Carregar X_train.txt ou X_test.txt em um DataFrame
   X = pd.read_csv(caminho_X, header=None, sep=" ", skipinitialspace=True)

   # Renomear as colunas de X usando os nomes em features
   X.columns = features

   # Adicionar o identificador do sujeito como uma nova coluna no DataFrame
   X['individuo'] = subject

   # Criar um índice duplo usando a ordem dos dados e o identificador do sujeito
   X.set_index(['individuo'], inplace=True)

   # Criar o diretório ./output/ se não existir
   os.makedirs("./output/", exist_ok=True)

   # Salvar os DataFrames em arquivos CSV
   X.to_csv(f"./output/X_{tipo}.csv", index=True)

   return X




caminho_features = r"C:\Users\Matheus\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\features.txt"
caminho_subject = r"C:\Users\Matheus\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\train\subject_train.txt"
caminho_X = r"C:\Users\Matheus\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\train\X_train.txt"

X_train = processa_dados_leitura_e_salvamento(caminho_features=caminho_features, caminho_subject=caminho_subject,
                                                               caminho_X=caminho_X)
X_train.head()

C:\Users\Matheus\AppData\Local\Temp\ipykernel_1324\1022191355.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  features = pd.read_csv(caminho_features, delim_whitespace=True, header=None, usecols=[1]).squeeze()


Var,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
individuo,,,,,,,,,,,,,,,,,,,,,
1,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
1,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
1,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
1,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


In [37]:
caminho_subject=r"C:\Users\Matheus\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\test\subject_test.txt"
caminho_X = r"C:\Users\Matheus\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\test\X_test.txt"
X_test = processa_dados_leitura_e_salvamento(caminho_features=caminho_features, caminho_subject=caminho_subject,
                                                             caminho_X=caminho_X, tipo="test")

C:\Users\Matheus\AppData\Local\Temp\ipykernel_1324\1022191355.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  features = pd.read_csv(caminho_features, delim_whitespace=True, header=None, usecols=[1]).squeeze()


In [38]:
y_train = pd.read_csv(r"C:\Users\Matheus\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\train\y_train.txt", header=None)
y_test = pd.read_csv(r"C:\Users\Matheus\Downloads\human+activity+recognition+using+smartphones\UCI HAR Dataset\UCI HAR Dataset\test\y_test.txt", header=None)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [47]:
%%time

clf = DecisionTreeClassifier(ccp_alpha=0.001).fit(X_train, y_train)


CPU times: total: 5.56 s
Wall time: 6.63 s


In [51]:
%%time

accuracy = clf.score(X_test, y_test)
accuracy

CPU times: total: 15.6 ms
Wall time: 18 ms


0.8802171700033933

### O tempo total foi de cerca de 6 segundos, arredondando. Um tempo bastante aceitável. Isso se deve provavelmente ao fato de que restringimos completamente o ccp.alpha, não avaliando várias possibilidades...

## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [99]:
%%time
prcomp = PCA().fit(X_train)
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)
pc_treino.shape

CPU times: total: 1.05 s
Wall time: 524 ms


(7352, 561)

In [113]:
%%time

n=1
colunas = ['cp' + str(x+1) for x in range(0,n)]
df_treino = pd.DataFrame(pc_treino[:,:n], columns = colunas)
df_treino

df_teste = pd.DataFrame(pc_teste[:,:n], columns = colunas)
clf1 = DecisionTreeClassifier(ccp_alpha=0.001, random_state=40028922).fit(df_treino, y_train)
score1 = clf1.score(df_teste, y_test)
score1


CPU times: total: 31.2 ms
Wall time: 74 ms


0.45707499151679676

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [103]:
%%time

numbers=list(range(1, 51, 2))
scores = []
tree_number = []
for n in numbers:
    colunas = ['cp' + str(x+1) for x in range(0,n)]
    df_treino = pd.DataFrame(pc_treino[:,:n], columns = colunas)
    df_teste = pd.DataFrame(pc_teste[:,:n], columns = colunas)
    clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=40028922).fit(df_treino, y_train)
    tree_number.append(n)
    score = clf.score(df_teste, y_test)
    scores.append(score)

CPU times: total: 7.31 s
Wall time: 8.29 s


In [105]:

for n in range(0, len(scores)):
    print(f'A acurácia da árvore {n+1} foi {scores[n]}. Esta árvore tinha {numbers[n]} elementos')


A acurácia da árvore 1 foi 0.45707499151679676. Esta árvore tinha 1 elementos
A acurácia da árvore 2 foi 0.7543264336613505. Esta árvore tinha 3 elementos
A acurácia da árvore 3 foi 0.7889379029521547. Esta árvore tinha 5 elementos
A acurácia da árvore 4 foi 0.7770614183915847. Esta árvore tinha 7 elementos
A acurácia da árvore 5 foi 0.8038683406854428. Esta árvore tinha 9 elementos
A acurácia da árvore 6 foi 0.8242280285035629. Esta árvore tinha 11 elementos
A acurácia da árvore 7 foi 0.8150661689854088. Esta árvore tinha 13 elementos
A acurácia da árvore 8 foi 0.8140481845945029. Esta árvore tinha 15 elementos
A acurácia da árvore 9 foi 0.8198167628096369. Esta árvore tinha 17 elementos
A acurácia da árvore 10 foi 0.8198167628096369. Esta árvore tinha 19 elementos
A acurácia da árvore 11 foi 0.8198167628096369. Esta árvore tinha 21 elementos
A acurácia da árvore 12 foi 0.8204954190702409. Esta árvore tinha 23 elementos
A acurácia da árvore 13 foi 0.8238887003732609. Esta árvore tinha

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

### O que podemos concluir é que adicionar variáveis foi efetivo para aumentar a acuracia até atingir o número de 6 componentes. A partir daí, o acréscimo de mais componentes se demonstrou de impacto neutro, ou até negativo sobre a acurácia...

### No entanto, provavelmente essa acuracia ainda poderia ser melhorada usando, por exemplo, um método Random Forest, ou até mesmo apenas um GridSearch mais extensivo...

### O tempo de processamento aumentou multiplas vezes, como o tempo inicial era da ordem dos milisegundos, não foi impactante na vida real. Teria sido impactante, no entanto, se a primeira etapa fosse mais custosa computacionalmente falando. O aumento foi de 7130/31 vezes. Ou seja, um aumento de 230 vezes. Caso o nosso processo inicial demorasse 1 minuto, o processo final demoraria mais de 3 horas!